In [ ]:
import numpy as np
import skfuzzy as fuzz
import matplotlib.pyplot as plt

# --- PART 1: THE MAMDANI FUZZY SYSTEM ---
class GridEarlyWarning_Mamdani:
    def __init__(self):
        self._setup_domains()
        self._setup_membership_functions()
        
    def _setup_domains(self):
        # Time: 0 to 24 hours
        self.x_time = np.arange(0, 24.1, 0.1)
        # Day: 0 (Monday) to 6 (Sunday)
        self.x_day = np.arange(0, 7, 0.1) 
        # Weather: 0 to 10 mm (Rainfall)
        self.x_weather = np.arange(0, 10.1, 0.1)
        # Risk Output: 0 to 100%
        self.x_risk = np.arange(0, 101, 1)

    def _setup_membership_functions(self):
        # 1. TIME (The Heartbeat)
        # Night: 00:00 - 07:00
        self.time_night = fuzz.trapmf(self.x_time, [0, 0, 6, 8]) 
        # Ramp_Up: 08:00 - 10:00 (The Danger Zone)
        self.time_ramp = fuzz.trimf(self.x_time, [7, 9, 11])
        # Peak: 11:00 - 18:00
        self.time_peak = fuzz.trapmf(self.x_time, [10, 12, 17, 19])
        # Evening: 19:00 - 23:00
        self.time_evening = fuzz.trapmf(self.x_time, [18, 20, 24, 24])

        # 2. DAY (The Context)
        # Weekday: Mon(0) to Fri(4). Starts dropping Fri evening.
        self.day_weekday = fuzz.trapmf(self.x_day, [0, 0, 4, 5])
        # Weekend: Sat(5) to Sun(6).
        self.day_weekend = fuzz.trapmf(self.x_day, [4, 5, 6, 6])

        # 3. WEATHER (The Bias)
        # Clear: < 0.5mm
        self.weather_clear = fuzz.sigmf(self.x_weather, 0.5, -5) # Sigmoid dropping at 0.5
        # Heavy Rain: > 0.5mm
        self.weather_heavy = fuzz.sigmf(self.x_weather, 0.5, 5)  # Sigmoid rising at 0.5

        # 4. OUTPUT: RISK LEVEL
        self.risk_low = fuzz.trimf(self.x_risk, [0, 0, 50])
        self.risk_med = fuzz.trimf(self.x_risk, [25, 50, 75])
        self.risk_high = fuzz.trimf(self.x_risk, [50, 100, 100])

    def evaluate(self, time_input, day_input, rain_input, visualize=False):
        # --- FUZZIFICATION ---
        # Time
        mu_night = fuzz.interp_membership(self.x_time, self.time_night, time_input)
        mu_ramp = fuzz.interp_membership(self.x_time, self.time_ramp, time_input)
        mu_peak = fuzz.interp_membership(self.x_time, self.time_peak, time_input)
        mu_evening = fuzz.interp_membership(self.x_time, self.time_evening, time_input)
        
        # Day
        mu_weekday = fuzz.interp_membership(self.x_day, self.day_weekday, day_input)
        mu_weekend = fuzz.interp_membership(self.x_day, self.day_weekend, day_input)
        
        # Weather
        mu_clear = fuzz.interp_membership(self.x_weather, self.weather_clear, rain_input)
        mu_heavy = fuzz.interp_membership(self.x_weather, self.weather_heavy, rain_input)

        # --- RULE EVALUATION (Based on your 9 Rules) ---
        
        # CATEGORY 1: HIGH RISK (Critical)
        # R1: Pagi (Ramp) AND Weekday -> High
        r1 = np.fmin(mu_ramp, mu_weekday)
        # R2: Pagi (Ramp) AND Heavy Rain -> High
        r2 = np.fmin(mu_ramp, mu_heavy)
        # R3: Siang (Peak) AND Weekday AND Heavy Rain -> High
        r3 = np.fmin(mu_peak, np.fmin(mu_weekday, mu_heavy))
        
        # Combine High Rules (OR logic)
        active_high = np.fmax(r1, np.fmax(r2, r3))
        
        # CATEGORY 2: MEDIUM RISK (Caution)
        # R4: Pagi (Ramp) AND Weekend AND Clear -> Med
        r4 = np.fmin(mu_ramp, np.fmin(mu_weekend, mu_clear))
        # R5: Siang (Peak) AND Weekday AND Clear -> Med
        r5 = np.fmin(mu_peak, np.fmin(mu_weekday, mu_clear))
        # R6: Siang (Peak) AND Weekend AND Heavy Rain -> Med
        r6 = np.fmin(mu_peak, np.fmin(mu_weekend, mu_heavy))
        
        # Combine Med Rules
        active_med = np.fmax(r4, np.fmax(r5, r6))
        
        # CATEGORY 3: LOW RISK (Safe)
        # R7: Night -> Low
        r7 = mu_night
        # R8: Sore (Evening) -> Low
        r8 = mu_evening
        # R9: Siang (Peak) AND Weekend AND Clear -> Low
        r9 = np.fmin(mu_peak, np.fmin(mu_weekend, mu_clear))
        
        # Combine Low Rules
        active_low = np.fmax(r7, np.fmax(r8, r9))
        
        # --- AGGREGATION & DEFUZZIFICATION ---
        # Clip the shapes
        out_high = np.fmin(active_high, self.risk_high)
        out_med = np.fmin(active_med, self.risk_med)
        out_low = np.fmin(active_low, self.risk_low)
        
        # Combine
        aggregated = np.fmax(out_low, np.fmax(out_med, out_high))
        
        # Calculate Centroid (Result)
        if np.sum(aggregated) == 0:
            result = 0
        else:
            result = fuzz.defuzz(self.x_risk, aggregated, 'centroid')
            
        return result

# --- PART 2: THE BAYESIAN NETWORK (Inference Engine) ---
class GridBayesNet:
    def __init__(self):
        # Conditional Probability Table (CPT)
        # This answers: "Given the Fuzzy Risk Level, what is the probability of ACTUAL Failure?"
        # Format: { 'Context_State': Probability_of_Failure }
        self.cpt_failure = {
            'Low': 0.05,    # 5% chance of failure if Fuzzy says Low
            'Medium': 0.40, # 40% chance of failure if Fuzzy says Medium
            'High': 0.95    # 95% chance of failure if Fuzzy says High
        }
        
    def infer(self, fuzzy_score):
        """
        Takes the continuous Fuzzy Score (0-100) and updates the Bayesian Belief.
        """
        # Step 1: Discretize the Fuzzy Score into Evidence
        # (This links the two systems)
        if fuzzy_score < 40:
            evidence_state = 'Low'
        elif fuzzy_score < 75:
            evidence_state = 'Medium'
        else:
            evidence_state = 'High'
            
        # Step 2: Query the CPT (Bayesian Update)
        # In a full net, this would be P(Failure | Evidence)
        prob_failure = self.cpt_failure[evidence_state]
        
        return evidence_state, prob_failure

# --- PART 3: MAIN EXECUTION ---
def run_early_warning_system(time_val, day_val, rain_val):
    print(f"\n--- SENSOR INPUT: Time={time_val:.1f}h, DayIndex={day_val}, Rain={rain_val}mm ---")
    
    # 1. Mamdani Processing
    fuzzy_system = GridEarlyWarning_Mamdani()
    risk_score = fuzzy_system.evaluate(time_val, day_val, rain_val)
    print(f"1. [Fuzzy Logic] Calculated Risk Score: {risk_score:.2f}/100")
    
    # 2. Bayesian Processing
    bayes_net = GridBayesNet()
    context, prob_fail = bayes_net.infer(risk_score)
    print(f"2. [Bayes Net] Evidence Context: '{context}'")
    print(f"3. [RESULT] Final Probability of Overload: {prob_fail*100:.1f}%")
    
    # Advice based on result
    if prob_fail > 0.8:
        print(">>> ADVICE: CRITICAL ALERT. START BACKUP GENERATORS IMMEDIATELY.")
    elif prob_fail > 0.3:
        print(">>> ADVICE: CAUTION. Put backup generators on standby.")
    else:
        print(">>> ADVICE: System Stable. Normal operation.")

# --- TEST CASES ---
# Case A: Monday Morning (09:00), No Rain -> EXPECT HIGH
run_early_warning_system(9.0, 0, 0.0) 

# Case B: Sunday Morning (09:00), No Rain -> EXPECT MEDIUM (Weekend Filter)
run_early_warning_system(9.0, 6, 0.0)

# Case C: Sunday Morning (09:00), HEAVY RAIN -> EXPECT HIGH (Weather Bias)
run_early_warning_system(9.0, 6, 5.0)


--- SENSOR INPUT: Time=9.0h, DayIndex=0, Rain=0.0mm ---
1. [Fuzzy Logic] Calculated Risk Score: 83.33/100
2. [Bayes Net] Evidence Context: 'High'
3. [RESULT] Final Probability of Overload: 95.0%
>>> ADVICE: CRITICAL ALERT. START BACKUP GENERATORS IMMEDIATELY.

--- SENSOR INPUT: Time=9.0h, DayIndex=6, Rain=0.0mm ---
1. [Fuzzy Logic] Calculated Risk Score: 52.72/100
2. [Bayes Net] Evidence Context: 'Medium'
3. [RESULT] Final Probability of Overload: 40.0%
>>> ADVICE: CAUTION. Put operators on standby.

--- SENSOR INPUT: Time=9.0h, DayIndex=6, Rain=5.0mm ---
1. [Fuzzy Logic] Calculated Risk Score: 83.33/100
2. [Bayes Net] Evidence Context: 'High'
3. [RESULT] Final Probability of Overload: 95.0%
>>> ADVICE: CRITICAL ALERT. START BACKUP GENERATORS IMMEDIATELY.


In [4]:
import numpy as np
import skfuzzy as fuzz
from probability import BayesNet, enumeration_ask, T, F # Importing directly from your file

# --- PART 1: MAMDANI FUZZY SYSTEM (The Intelligence) ---
class GridEarlyWarning_Mamdani:
    def __init__(self):
        self._setup_domains()
        self._setup_membership_functions()
        
    def _setup_domains(self):
        self.x_time = np.arange(0, 24.1, 0.1)
        self.x_day = np.arange(0, 7, 0.1) 
        self.x_weather = np.arange(0, 10.1, 0.1)
        self.x_risk = np.arange(0, 101, 1)

    def _setup_membership_functions(self):
        # Definitions matching our analysis
        self.time_night = fuzz.trapmf(self.x_time, [0, 0, 6, 8]) 
        self.time_ramp = fuzz.trimf(self.x_time, [7, 9, 11])
        self.time_peak = fuzz.trapmf(self.x_time, [10, 12, 17, 19])
        self.time_evening = fuzz.trapmf(self.x_time, [18, 20, 24, 24])

        self.day_weekday = fuzz.trapmf(self.x_day, [0, 0, 4, 5])
        self.day_weekend = fuzz.trapmf(self.x_day, [4, 5, 6, 6])

        self.weather_clear = fuzz.sigmf(self.x_weather, 0.5, -5)
        self.weather_heavy = fuzz.sigmf(self.x_weather, 0.5, 5)

        self.risk_low = fuzz.trimf(self.x_risk, [0, 0, 50])
        self.risk_med = fuzz.trimf(self.x_risk, [25, 50, 75])
        self.risk_high = fuzz.trimf(self.x_risk, [50, 100, 100])

    def evaluate(self, time_input, day_input, rain_input):
        # Fuzzification
        mu_ramp = fuzz.interp_membership(self.x_time, self.time_ramp, time_input)
        mu_peak = fuzz.interp_membership(self.x_time, self.time_peak, time_input)
        mu_night = fuzz.interp_membership(self.x_time, self.time_night, time_input)
        mu_evening = fuzz.interp_membership(self.x_time, self.time_evening, time_input)
        
        mu_weekday = fuzz.interp_membership(self.x_day, self.day_weekday, day_input)
        mu_weekend = fuzz.interp_membership(self.x_day, self.day_weekend, day_input)
        
        mu_heavy = fuzz.interp_membership(self.x_weather, self.weather_heavy, rain_input)
        mu_clear = fuzz.interp_membership(self.x_weather, self.weather_clear, rain_input)

        # Rule Base Evaluation (The 9 Rules)
        # 1. High Risk
        r1 = np.fmin(mu_ramp, mu_weekday)
        r2 = np.fmin(mu_ramp, mu_heavy)
        r3 = np.fmin(mu_peak, np.fmin(mu_weekday, mu_heavy))
        active_high = np.fmax(r1, np.fmax(r2, r3))
        
        # 2. Medium Risk
        r4 = np.fmin(mu_ramp, np.fmin(mu_weekend, mu_clear))
        r5 = np.fmin(mu_peak, np.fmin(mu_weekday, mu_clear))
        r6 = np.fmin(mu_peak, np.fmin(mu_weekend, mu_heavy))
        active_med = np.fmax(r4, np.fmax(r5, r6))
        
        # 3. Low Risk
        r7 = mu_night
        r8 = mu_evening
        r9 = np.fmin(mu_peak, np.fmin(mu_weekend, mu_clear))
        active_low = np.fmax(r7, np.fmax(r8, r9))
        
        # Aggregation & Defuzzification
        out_high = np.fmin(active_high, self.risk_high)
        out_med = np.fmin(active_med, self.risk_med)
        out_low = np.fmin(active_low, self.risk_low)
        
        aggregated = np.fmax(out_low, np.fmax(out_med, out_high))
        
        if np.sum(aggregated) == 0: return 0
        return fuzz.defuzz(self.x_risk, aggregated, 'centroid')

# --- PART 2: BAYESIAN NETWORK SETUP (Using probability.py) ---

def make_grid_bayes_net():
    """
    Constructs the BayesNet using the class from probability.py.
    Since the library is Boolean-only, we map 3 Fuzzy States to 2 Boolean Variables.
    
    States Logic:
    - Low Risk:    Elevated=False, Critical=False
    - Medium Risk: Elevated=True,  Critical=False
    - High Risk:   Elevated=True,  Critical=True
    """
    return BayesNet([
        ('Risk_Elevated', '', 0.5), # Prior doesn't matter, we observe this
        ('Risk_Critical', '', 0.5), # Prior doesn't matter, we observe this
        
        # The 'Overload' node depends on the two boolean flags.
        # Format: {(Parent1, Parent2): Probability_of_True}
        ('Overload', 'Risk_Elevated Risk_Critical', {
            (T, T): 0.95,  # High Context (Both True) -> 95% Failure Prob
            (T, F): 0.40,  # Medium Context (Elevated only) -> 40% Failure Prob
            (F, F): 0.05,  # Low Context (Neither) -> 5% Failure Prob
            (F, T): 0.95   # Impossible state (Critical but not Elevated), treat as High
        })
    ])

def map_fuzzy_to_evidence(score):
    """
    Maps the continuous Fuzzy Score (0-100) to the Boolean Evidence dictionary.
    """
    evidence = {}
    
    # Logic for Risk_Elevated (Is it at least Medium?)
    if score >= 40:
        evidence['Risk_Elevated'] = T
    else:
        evidence['Risk_Elevated'] = F
        
    # Logic for Risk_Critical (Is it High?)
    if score >= 75:
        evidence['Risk_Critical'] = T
    else:
        evidence['Risk_Critical'] = F
        
    return evidence

# --- PART 3: MAIN EXECUTION FLOW ---

def run_system_mamdani(time_val, day_val, rain_val):
    print(f"\n--- SENSOR INPUT: Time={time_val}, Day={day_val}, Rain={rain_val} ---")
    
    # 1. Fuzzy Inference
    fuzzy_logic = GridEarlyWarning_Mamdani()
    risk_score = fuzzy_logic.evaluate(time_val, day_val, rain_val)
    print(f"[1] Fuzzy Risk Score: {risk_score:.2f} / 100")
    
    # 2. Map to Boolean Evidence for AIMA BayesNet
    evidence = map_fuzzy_to_evidence(risk_score)
    print(f"[2] Evidence for BayesNet: {evidence}")
    
    # 3. Bayesian Inference using enumeration_ask
    bn = make_grid_bayes_net()
    
    # Query: What is the probability of 'Overload'?
    result_dist = enumeration_ask('Overload', evidence, bn)
    
    # result_dist is a ProbDist object. We access True (Overload=True)
    prob_failure = result_dist[T]
    
    print(f"[3] Final Probability of Overload: {prob_failure*100:.1f}%")
    
    # Decision
    if prob_failure > 0.80:
        print(">>> ACTION: CRITICAL WARNING. DISPATCH GENERATORS.")
    elif prob_failure > 0.30:
        print(">>> ACTION: STANDBY. MONITOR GRID.")
    else:
        print(">>> ACTION: SYSTEM NORMAL.")

# --- TEST SCENARIOS ---

# Case A: Monday Morning Ramp (09:00), Weekday -> High Risk
run_system_mamdani(9.0, 0, 0.0)

# Case B: Sunday Morning Ramp (09:00), Weekend -> Medium Risk
run_system_mamdani(9.0, 6, 0.0)

# Case C: _mamdaniNight Time (02:00) -> Low Risk
run_system_mamdani(2.0, 0, 0.0)


--- SENSOR INPUT: Time=9.0, Day=0, Rain=0.0 ---
[1] Fuzzy Risk Score: 83.33 / 100
[2] Evidence for BayesNet: {'Risk_Elevated': True, 'Risk_Critical': True}
[3] Final Probability of Overload: 95.0%
>>> ACTION: CRITICAL WARNING. DISPATCH GENERATORS.

--- SENSOR INPUT: Time=9.0, Day=6, Rain=0.0 ---
[1] Fuzzy Risk Score: 52.72 / 100
[2] Evidence for BayesNet: {'Risk_Elevated': True, 'Risk_Critical': False}
[3] Final Probability of Overload: 40.0%
>>> ACTION: STANDBY. MONITOR GRID.

--- SENSOR INPUT: Time=2.0, Day=0, Rain=0.0 ---
[1] Fuzzy Risk Score: 16.67 / 100
[2] Evidence for BayesNet: {'Risk_Elevated': False, 'Risk_Critical': False}
[3] Final Probability of Overload: 5.0%
>>> ACTION: SYSTEM NORMAL.


<h1> tsukamoto yeah

In [8]:
class TsukamotoFuzzySystem:
    def __init__(self):
        self._setup_domains()
        self._setup_membership_functions()

    def _setup_domains(self):
        self.x_time = np.arange(0, 24.1, 0.1)
        self.x_day = np.arange(0, 7, 0.1) 
        self.x_weather = np.arange(0, 10.1, 0.1)
        self.x_risk = np.arange(0, 101, 1)

    def _setup_membership_functions(self):
        # Same MF definitions as Mamdani
        self.time_night = fuzz.trapmf(self.x_time, [0, 0, 6, 8]) 
        self.time_ramp = fuzz.trimf(self.x_time, [7, 9, 11])
        self.time_peak = fuzz.trapmf(self.x_time, [10, 12, 17, 19])
        self.time_evening = fuzz.trapmf(self.x_time, [18, 20, 24, 24])

        self.day_weekday = fuzz.trapmf(self.x_day, [0, 0, 4, 5])
        self.day_weekend = fuzz.trapmf(self.x_day, [4, 5, 6, 6])

        self.weather_clear = fuzz.sigmf(self.x_weather, 0.5, -5)
        self.weather_heavy = fuzz.sigmf(self.x_weather, 0.5, 5)

    # --- MISSING INVERSE FUNCTIONS (CRITICAL FOR TSUKAMOTO) ---
    def _inv_low(self, alpha):
        # Line going DOWN from 1.0 (at 0) to 0.0 (at 50)
        # z = 50 - (alpha * 50)
        return 50 - (alpha * 50)

    def _inv_med(self, alpha):
        # Simplified: Return center of Medium
        return 50.0 

    def _inv_high(self, alpha):
        # Line going UP from 0.0 (at 50) to 1.0 (at 100)
        # z = (alpha * 50) + 50
        return (alpha * 50) + 50

    def evaluate(self, time_input, day_input, rain_input):
        # Fuzzification
        mu_ramp = fuzz.interp_membership(self.x_time, self.time_ramp, time_input)
        mu_peak = fuzz.interp_membership(self.x_time, self.time_peak, time_input)
        mu_night = fuzz.interp_membership(self.x_time, self.time_night, time_input)
        mu_evening = fuzz.interp_membership(self.x_time, self.time_evening, time_input)
        
        mu_weekday = fuzz.interp_membership(self.x_day, self.day_weekday, day_input)
        mu_weekend = fuzz.interp_membership(self.x_day, self.day_weekend, day_input)
        
        mu_heavy = fuzz.interp_membership(self.x_weather, self.weather_heavy, rain_input)
        mu_clear = fuzz.interp_membership(self.x_weather, self.weather_clear, rain_input)

        # Rule Base
        # High Risk
        r1 = np.fmin(mu_ramp, mu_weekday)
        r2 = np.fmin(mu_ramp, mu_heavy)
        r3 = np.fmin(mu_peak, np.fmin(mu_weekday, mu_heavy))
        
        # Medium Risk
        r4 = np.fmin(mu_ramp, np.fmin(mu_weekend, mu_clear))
        r5 = np.fmin(mu_peak, np.fmin(mu_weekday, mu_clear))
        r6 = np.fmin(mu_peak, np.fmin(mu_weekend, mu_heavy))
        
        # Low Risk
        r7 = mu_night
        r8 = mu_evening
        r9 = np.fmin(mu_peak, np.fmin(mu_weekend, mu_clear))
        
        # Tsukamoto Aggregation
        alpha_list, z_list = [], []

        # High Rules
        for alpha in [r1, r2, r3]:
            if alpha > 0:
                alpha_list.append(alpha)
                z_list.append(self._inv_high(alpha))
        
        # Medium Rules
        for alpha in [r4, r5, r6]:
            if alpha > 0:
                alpha_list.append(alpha)
                z_list.append(self._inv_med(alpha))
        
        # Low Rules
        for alpha in [r7, r8, r9]:
            if alpha > 0:
                alpha_list.append(alpha)
                z_list.append(self._inv_low(alpha))

        alpha_arr = np.array(alpha_list)
        z_arr = np.array(z_list)

        if np.sum(alpha_arr) == 0:
            return 0.0
        else:
            return np.sum(alpha_arr * z_arr) / np.sum(alpha_arr)
        

def run_system_tsukamoto(time_val, day_val, rain_val):
    print(f"\n--- [TSUKAMOTO] SENSOR INPUT: Time={time_val}, Day={day_val}, Rain={rain_val} ---")
    
    # 1. Fuzzy Inference (CORRECTED CLASS INSTANCE)
    fuzzy_logic = TsukamotoFuzzySystem() # <--- FIXED HERE
    risk_score = fuzzy_logic.evaluate(time_val, day_val, rain_val)
    print(f"[1] Tsukamoto Risk Score: {risk_score:.2f} / 100")
    
    # 2. Map to Evidence
    evidence = map_fuzzy_to_evidence(risk_score)
    print(f"[2] Evidence for BayesNet: {evidence}")
    
    # 3. Bayesian Inference
    bn = make_grid_bayes_net()
    result_dist = enumeration_ask('Overload', evidence, bn)
    prob_failure = result_dist[T]
    
    print(f"[3] Final Probability of Overload: {prob_failure*100:.1f}%")



# Case A: Monday Morning Ramp (09:00), Weekday -> High Risk
run_system_tsukamoto(9.0, 0, 0.0)

# Case B: Sunday Morning Ramp (09:00), Weekend -> Medium Risk
run_system_tsukamoto(9.0, 6, 0.0)

# Case C: _mamdaniNight Time (02:00) -> Low Risk
run_system_tsukamoto(2.0, 0, 0.0)



--- [TSUKAMOTO] SENSOR INPUT: Time=9.0, Day=0, Rain=0.0 ---
[1] Tsukamoto Risk Score: 96.74 / 100
[2] Evidence for BayesNet: {'Risk_Elevated': True, 'Risk_Critical': True}
[3] Final Probability of Overload: 95.0%

--- [TSUKAMOTO] SENSOR INPUT: Time=9.0, Day=6, Rain=0.0 ---
[1] Tsukamoto Risk Score: 50.29 / 100
[2] Evidence for BayesNet: {'Risk_Elevated': True, 'Risk_Critical': False}
[3] Final Probability of Overload: 40.0%

--- [TSUKAMOTO] SENSOR INPUT: Time=2.0, Day=0, Rain=0.0 ---
[1] Tsukamoto Risk Score: 0.00 / 100
[2] Evidence for BayesNet: {'Risk_Elevated': False, 'Risk_Critical': False}
[3] Final Probability of Overload: 5.0%
